# Introduction

In this tutorial, we will demonstrate how to use **Enclaves** in PySyft to securely perform computations among assets from multiple domains. We will cover the following workflows:

## Data Owners Workflow - Part 1 (Current notebook)
- Launch two domain nodes for providing data to the data scientist.
- Launch one enclave node for performing the secure computation using data from both the domain nodes.
- Upload datasets to both the domain nodes.
- Register an account for the data scientist in both the domain nodes.
- Register the enclave node with one of the domain nodes for discoverability by the data scientist.

## Data Scientist Workflow - Part 1 ([./01-ds-submit-project.ipynb](./01-ds-submit-project.ipynb))
- Find datasets across multiple domains.
- Find a suitable Enclave for performing the multi-party computation.
- Create a project containing code to perform multi-party computation.
- Send the project for review by the data owners.

## Data Owner Workflow - Part 2
- review code and approve
- TODO

## Data Scientist Workflow - Part 2
- TODO

## Enclave Workflow
- TODO

In [ ]:
# third party
import pytest
from recordlinkage.datasets import load_febrl4

# syft absolute
import syft as sy
from syft.abstract_node import NodeType
from syft.service.network.routes import HTTPNodeRoute
from syft.service.response import SyftAttributeError
from syft.service.response import SyftSuccess

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082
CANADA_ENCLAVE_PORT = 9083

# Launch nodes

We will begin by launching two domain nodes and an enclave node.

In [ ]:
canada_node = sy.orchestra.launch(
    name="canada-domain", port=CANADA_DOMAIN_PORT, dev_mode=True, reset=True
)
italy_node = sy.orchestra.launch(
    name="italy-domain", port=ITALY_DOMAIN_PORT, dev_mode=True, reset=True
)
canada_enclave = sy.orchestra.launch(
    name="canada-enclave",
    node_type=NodeType.ENCLAVE,
    port=CANADA_ENCLAVE_PORT,
    dev_mode=True,
    reset=True,
)

In [ ]:
do_canada_client = canada_node.login(email="info@openmined.org", password="changethis")
do_italy_client = italy_node.login(email="info@openmined.org", password="changethis")

assert do_canada_client.metadata.node_type == NodeType.DOMAIN
assert do_italy_client.metadata.node_type == NodeType.DOMAIN

# Upload datasets to both domains

In [ ]:
# Using public datasets from Freely Extensible Biomedical Record Linkage (Febrl) project
canada_census_data, italy_census_data = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data, italy_census_data],
    [do_canada_client, do_italy_client],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

In [ ]:
assert len(do_canada_client.datasets.get_all()) == 1
assert len(do_italy_client.datasets.get_all()) == 1

# Create account for data scientist on both the domains

In [ ]:
for client in [do_canada_client, do_italy_client]:
    res = client.register(
        name="Sheldon",
        email="sheldon@caltech.edu",
        password="changethis",
        password_verify="changethis",
    )
    assert isinstance(res, SyftSuccess)

# Register the enclave with Canada domain

In [ ]:
route = HTTPNodeRoute(host_or_ip="localhost", port=CANADA_ENCLAVE_PORT)
do_canada_client.enclaves.add(route=route)

In [ ]:
assert (len(do_canada_client.enclaves.get_all())) == 1
do_canada_client.enclaves.get_all()

In [ ]:
ds_canada_client = sy.login(
    email="sheldon@caltech.edu", password="changethis", port=CANADA_DOMAIN_PORT
)

In [ ]:
# Data scientist should not be able to add enclave to the domain
with pytest.raises(SyftAttributeError) as exc_info:
    ds_canada_client.enclaves.add(
        name="Dummy Enclave", route=HTTPNodeRoute(host_or_ip="localhost", port=9084)
    )
exc_info.value

In [ ]:
assert (len(ds_canada_client.enclaves.get_all())) == 1
ds_canada_client.enclaves.get_all()

# Cleanup local domain servers

In [ ]:
if canada_node.node_type.value == "python":
    canada_node.land()

if italy_node.node_type.value == "python":
    italy_node.land()

if canada_enclave.node_type.value == "python":
    canada_enclave.land()